# Q1

In [37]:
import pandas as pd

In [38]:
#downloading the data
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -O ./data/yellow_tripdata_2023-01.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet -O ./data/yellow_tripdata_2023-02.parquet

--2024-05-27 16:05:51--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.238.152, 18.239.238.119, 18.239.238.133, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.238.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘./data/yellow_tripdata_2023-01.parquet’

./data/yellow_tripd 100%[===================>]  45.46M   119MB/s    in 0.4s    

2024-05-27 16:05:51 (119 MB/s) - ‘./data/yellow_tripdata_2023-01.parquet’ saved [47673370/47673370]

--2024-05-27 16:05:52--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.238.133, 18.239.238.212, 18.239.238.119, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.23

In [39]:
#loading the january data
df_january = pd.read_parquet('data/yellow_tripdata_2023-01.parquet')

In [40]:
#number of columns
print(len(df_january.columns))

19


# Q2

In [41]:
# compute the durations

In [42]:
df_january.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [43]:
#compute durations and give the std of the durations
df_january["tpep_pickup_datetime"] = pd.to_datetime(df_january["tpep_pickup_datetime"])
df_january["tpep_dropoff_datetime"] = pd.to_datetime(df_january["tpep_dropoff_datetime"])
df_january["duration"]= df_january["tpep_dropoff_datetime"] - df_january["tpep_pickup_datetime"]
df_january["duration"] = df_january["duration"].apply(lambda d: d.total_seconds()/60)
print(f"durations std: {round(df_january.duration.std(),2)}")


durations std: 42.59


# Q3

In [44]:
#drop records where the duration was between 1 and 60 minutes (inclusive).
mask = (df_january.duration <= 60) & (df_january.duration >=1)
df_january= df_january[mask]

In [45]:
#print the fraction of remaining data
print(f"fraction of remaining data: {round(100* mask.sum()/len(mask),2)}%")

fraction of remaining data: 98.12%


# Q4

In [70]:
from sklearn.feature_extraction import DictVectorizer

In [71]:
#get the list of dictionaries from the dataframe
df_dicts = df_january[id_cols].to_dict(orient='records')
id_cols = ["PULocationID", "DOLocationID"]
df_january[id_cols]= df_january[id_cols].astype(str)

In [72]:
#Fit dictionary vectorizer and get feature matrix
dv = DictVectorizer()
X = dv.fit_transform(df_dicts)

In [73]:
#number of columns
print(f"number of columns of the features matrix: {X.shape[1]}")

number of columns of the features matrix: 515


# Q5

In [74]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [78]:
#train a linear regression model to predict the durations
lr = LinearRegression()
y = df_january["duration"].values
lr.fit(X,y)

LinearRegression()

In [81]:
# get the RMSE on train
y_pred = lr.predict(X)
print(f"RMSE on train data: {round(mean_squared_error(y_pred, y, squared=False), 2)}")

RMSE on train data: 7.65
